In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from multiinstance.density_ratio_em import DensityRatioEM,generateParams
from multiinstance.em import generateBags
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
NClusters = 5
Dim = 2
aucpn_range=  [.9,1.]
NBags = 100
NPos = 10
NUnlabeled = 25

In [ ]:
componentInfo = generateParams(NClusters,1,Dim,aucpn_range=aucpn_range)

In [ ]:
bags = generateBags(NBags,
                    pos_means=componentInfo.posMeans,
                    neg_means=componentInfo.negMeans,
                    pos_covs=componentInfo.posCovs,
                    neg_covs=componentInfo.negCovs,
                    NPos=NPos,NUnlabeled=NUnlabeled)

In [ ]:
drem = DensityRatioEM(bags,NClusters)

In [ ]:
cm = plt.get_cmap(name="tab20")

In [ ]:
U = np.zeros((0,2))
P = np.zeros((0,2))
colors = []
for b in drem.bags:
    U = np.concatenate((U,b.x_unlabeled))
    P = np.concatenate((P,b.X_pos))
    colors += [cm(i) for i in b.unlabeled_cluster_assignment]
#     plt.scatter(b.x_unlabeled[:,0], b.x_unlabeled[:,1],color=[cmap[i] for i in b.unlabeled_cluster_assignment])
#     plt.show()

In [ ]:
plt.scatter(U[:,0],U[:,1],color=["green"])
plt.scatter(P[:,0],P[:,1])

In [ ]:
np.mean(np.abs(drem.clusterAlphaHats - drem.trueclusterAlphaHats))

# True Distributions

In [ ]:
drem.run("true posterior",componentInfo)

In [ ]:
drem.getAUC()

In [ ]:
trueRatios = drem.bagRatios

# Estimated Posterior

In [ ]:
drem.run("estimated posterior", componentInfo)

In [ ]:
drem.getAUC()

### At least in this simple 2D setting, the ratio estimation does not appear to be the issue

In [ ]:
plt.scatter(np.log(np.concatenate([np.concatenate((l)) for l in drem.bagRatios])),
            np.log(np.concatenate([np.concatenate(l) for l in trueRatios])))
plt.plot([-10,10],[-10,10],color="black")

### Again, using the true cluster alpha hats then proceeding with the EM algorithm still results in noise in final bag level posteriors. Below, the true cluster alpha hats are assigned then the model proceeds with the EM algorithm. The final AUC is still less than that at the cluster level

In [ ]:
drem.clusterAlphaHats = drem.trueclusterAlphaHats

In [ ]:
drem.run("estimated posterior",componentInfo)

In [ ]:
drem.getAUC()

### Below is the result when using the true bag/cluster combination class priors.

In [ ]:
drem.eta = drem.trueEta

In [ ]:
drem.getAUC()

In [ ]:
drem.EM(NIters=100000)

In [ ]:
drem.getAUC()